# Домашнее задание

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**Ответ:**

- Mean average precision
- Normalized Discounted Cumulative Gain
- Mean reciprocal rank
- Метрики на основе ранговой корреляции
- Метрики на основе каскадной модели поведения

https://habr.com/ru/company/econtenta/blog/303458/

---

**2) Доделать все функции, где стоит комментарий "сделать дома"**

In [2]:
# Загрузка необходимых библиотек и данных

import numpy as np

recommended_l = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_l = [521, 32, 143, 991]

In [5]:
# Функция recall

def recall(recommended_l, bought_l):
    
    bought_l = np.array(bought_l)
    recommended_l = np.array(recommended_l)
    flags = np.isin(bought_l, recommended_l)
    
    return flags.sum() / len(bought_l)

# Функция recall@K

def recall_at_k(recommended_l, bought_l, k = 5):
    
    flags = np.isin(np.array(bought_l), np.array(recommended_l)[:k])
    recall = flags.sum() / len(np.array(bought_l))
    
    return recall

In [6]:
# Проверка

recall(recommended_l, bought_l), recall_at_k(recommended_l, bought_l)

(0.5, 0.5)

In [7]:
# Функция money_recall@K

def money_recall_at_k(recommended_l, bought_l, prices_recommended, prices_bought, k = 5):
    
    bought_l = np.array(bought_l)
    prices_bought = np.array(prices_bought)
    recommended_l = np.array(recommended_l)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_l, bought_l)
    money_recall = np.dot(flags, prices_recommended)/prices_bought.sum()
    
    return money_recall

In [10]:
# Загрузка данных

prices_recommended = [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]
prices_bought = [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113]

In [11]:
# Проверка

recall(recommended_l, bought_l), money_recall_at_k(recommended_l, bought_l, prices_recommended, prices_bought)

(0.5, 0.14593817397555714)

In [12]:
# Функция precision

def precision(recommended_l, bought_l):
    
    bought_l = np.array(bought_l)
    recommended_l = np.array(recommended_l)
    flags = np.isin(bought_l, recommended_l)
    
    return flags.sum() / len(recommended_l)

# Функция precision@K

def precision_at_k(recommended_l, bought_l, k = 5):
    
    return precision(recommended_l[:k], bought_l)

# Функция ap_k

def ap_k(recommended_l, bought_l, k = 5):
    
    bought_l = np.array(bought_l)
    recommended_l = np.array(recommended_l)[:k]    
    relevant_indexes = np.nonzero(np.isin(recommended_l, bought_l))[0]
    
    if len(relevant_indexes) == 0:
        
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    for index_relevant in relevant_indexes:
        
        print(precision_at_k(recommended_l, bought_l, k = index_relevant + 1) )
        
    sum_ = sum([precision_at_k(recommended_l, bought_l, k = index_relevant + 1) for index_relevant in relevant_indexes])
    
    return sum_ / amount_relevant

# Функция map_k

def map_k(recommended_l, bought_l, k = 5):
    
    amount_user = len(bought_l)
    list_ap_k = [ap_k(recommended_l[i], bought_l[i], k) for i in np.arange(amount_user)]    
    sum_ap_k = sum(list_ap_k)  
    result = sum_ap_k / amount_user
    
    return result

In [13]:
# Загрузка данных

recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],
                       [143, 156, 991, 43, 11],
                       [1, 2]
                      ]

In [14]:
# Проверка

map_k(recommended_list_3_users, bought_list_3_users)

1.0


0.3333333333333333

In [15]:
# Функция inverse_rank

def inverse_rank(recommended_l, bought_l, k = 1):
    
    rank = [(1/(i+1)) for i in range(len(recommended_l[:k])) for j in range(len(bought_l)) if recommended_l[i] == bought_l[j]]
    
    if len(rank) > 0:
        
        return rank[0]
    
    else:
        
        return 0
    
# Функция reciprocal_rank

def reciprocal_rank(recommended_list_users, bought_list_users, k=1):
    
    rec_rank = []
    
    for i in range(len(bought_list_users)):
        
        rec_rank.append(inverse_rank(recommended_list_users[i], bought_list_users[i], k))
        
    rec_rank
    
    return rec_rank

# Функция reciprocal_rank

def mean_reciprocal_rank(recommended_list_users, bought_list_users, k=1):
    
    rec_rank = reciprocal_rank(recommended_list_users, bought_list_users, k)

    mrr_at_k = np.sum(rec_rank) / len(bought_list_users)
    
    return mrr_at_k

In [16]:
# Проверка reciprocal

reciprocal_rank(recommended_list_3_users, bought_list_3_users, k = 10)

[1.0, 0, 0]

In [17]:
# Проверка mean_reciprocal

mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users, k = 10)

0.3333333333333333